In [17]:
trg_table_name = 'tgt_incr_emp'

StatementMeta(, 6c81eeeb-6932-49d5-9592-97a24c6616f7, 19, Finished, Available, Finished)

In [18]:
# Query the max hiredate
df = spark.sql("""
    SELECT MAX(hiredate) AS max_hiredate
    FROM dbo.src_incr_emp
""")

# Extract it into a Python variable
max_hiredate = df.collect()[0]['max_hiredate']
print(f"Max hiredate: {max_hiredate}")

StatementMeta(, 6c81eeeb-6932-49d5-9592-97a24c6616f7, 20, Finished, Available, Finished)

Max hiredate: 2020-09-05 08:06:00


In [25]:
# Read the max timestamp into a DataFrame
df1 = spark.sql("""
    SELECT MAX(last_load_date) AS last_load_date
    FROM dbo.watermarktable WHERE TableName = '{trg_table_name}'
""")

# Extract the value into a Python variable
last_load_date = df1.collect()[0]['last_load_date']
print(f"Last load date: {last_load_date}")

StatementMeta(, 6c81eeeb-6932-49d5-9592-97a24c6616f7, 27, Finished, Available, Finished)

Last load date: None


In [20]:
# only complete day

query = f"""
SELECT * FROM dbo.src_incr_emp
WHERE hiredate >= to_timestamp('{last_load_date}', 'yyyy-MM-dd HH:mm:ss')
  AND hiredate < to_timestamp('{max_hiredate}', 'yyyy-MM-dd HH:mm:ss')
"""

query

StatementMeta(, 6c81eeeb-6932-49d5-9592-97a24c6616f7, 22, Finished, Available, Finished)

"\nSELECT * FROM dbo.src_incr_emp\nWHERE hiredate >= to_timestamp('None', 'yyyy-MM-dd HH:mm:ss')\n  AND hiredate < to_timestamp('2020-09-05 08:06:00', 'yyyy-MM-dd HH:mm:ss')\n"

In [21]:
df = spark.sql(query)

df.write \
  .format("delta") \
  .mode("append") \
  .saveAsTable("dbo.tgt_incr_emp")

StatementMeta(, 6c81eeeb-6932-49d5-9592-97a24c6616f7, 23, Finished, Available, Finished)

In [22]:
spark.sql("SELECT * FROM dbo.tgt_incr_emp").show()

StatementMeta(, 6c81eeeb-6932-49d5-9592-97a24c6616f7, 24, Finished, Available, Finished)

+-----+-------------+-------------------+
|empid|        ename|           hiredate|
+-----+-------------+-------------------+
|    3|         John|2020-09-03 02:36:00|
|    4|         Mark|2020-09-04 03:21:00|
|    1|Myla RamReddy|2020-09-01 00:56:00|
|    2|   Anam Tarun|2020-09-02 05:23:00|
+-----+-------------+-------------------+



In [23]:
spark.sql(f"""
    UPDATE watermarktable
    SET last_load_date = to_timestamp('{max_hiredate}', 'yyyy-MM-dd HH:mm:ss')
    WHERE TableName = '{trg_table_name}'
""")

StatementMeta(, 6c81eeeb-6932-49d5-9592-97a24c6616f7, 25, Finished, Available, Finished)

DataFrame[num_affected_rows: bigint]

In [24]:
%%sql
select * from dbo.watermarktable

StatementMeta(, 6c81eeeb-6932-49d5-9592-97a24c6616f7, 26, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 2 fields>